### Парсер к проекту Ford vs Ferrari "Skillfactory.ru"

In [ ]:
import requests
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep
from bs4 import BeautifulSoup

In [ ]:
# подключу GoogleDisk
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./drive/MyDrive/db_cars_VAZ/0'

In [ ]:
classes = {
    0: ['https://auto.ru/cars/vaz/2170/used/body-wagon/?page=', 10], #Приора универсал
    1: ['https://auto.ru/cars/ford/focus/used/body-hatchback/?page=', 10], #Focus хэтчбэк
    5: ['https://auto.ru/cars/chevrolet/niva/used/?page=', 10], #Шнива будет Нива
    6: ['https://auto.ru/cars/vaz/kalina/used/?page=', 10], #Калина
    7: ['https://auto.ru/cars/vaz/2109/all/?page=', 20], #Девятки
    8: ['https://auto.ru/cars/volkswagen/passat/all/?year_to=2000/?page=', 10], #Старый passat
    9: ['https://auto.ru/cars/vaz/21099/all/?page=', 20] #99
}

In [ ]:
def download_image(url, class_id, file_name):
    directory = f'./drive/MyDrive/db_cars_VAZ/{class_id}'
    if not os.path.exists(directory):
        os.makedirs(directory)
#     file_name = 0
    while os.path.exists(f'{directory}/{file_name}.jpg'):
        file_name = file_name + 1
    # open in binary mode
    with open(f'{directory}/{file_name}.jpg', "wb") as file:
        # get request
        response = requests.get(url)
        # write to file
        file.write(response.content)

In [ ]:
for class_id in classes:
    file_name = 0
    print(f'Class - {class_id}')
    for page in range(1, classes[class_id][1]+1):
        print(f'Page - {page}/{classes[class_id][1]}')
        url = f'{classes[class_id][0]}{page}'
        print(url)
        response = requests.get(url)
        if response == -1:
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', class_='Link ListingItemTitle__link')
        for link in links:
            print(link.get('href'))
            response = requests.get(link.get('href'))
            if response == -1:
                continue
            soup_m = BeautifulSoup(response.text, 'html.parser')
            images = soup_m.find_all('img', class_='ImageGalleryDesktop__image')
            for image in images[:2]:# двух первых фоток достаточно, чтобы потом не чистить от мусора
                download_image(f"http:{image.get('src')}", class_id, file_name)
                file_name += 1
        sleep(5)

Class - 0
Page - 1/10
https://auto.ru/cars/vaz/2170/used/body-wagon/?page=1
https://auto.ru/cars/used/sale/vaz/2170/1105806679-a3fc87af/
https://auto.ru/cars/used/sale/vaz/2170/1106160037-dfb7c674/
https://auto.ru/cars/used/sale/vaz/2170/1105856307-5b32a00a/
https://auto.ru/cars/used/sale/vaz/2170/1106100882-92edf28f/
https://auto.ru/cars/used/sale/vaz/2170/1105716022-c6ee6e36/
https://auto.ru/cars/used/sale/vaz/2170/1106099480-222adfcf/
https://auto.ru/cars/used/sale/vaz/2170/1106172462-5ec30387/
https://auto.ru/cars/used/sale/vaz/2170/1106107562-dfea699f/
https://auto.ru/cars/used/sale/vaz/2170/1105861611-2dfb58fa/
https://auto.ru/cars/used/sale/vaz/2170/1106178879-b42e3ef1/
https://auto.ru/cars/used/sale/vaz/2170/1105829822-f0006a03/
https://auto.ru/cars/used/sale/vaz/2170/1106168803-ab0bec36/
https://auto.ru/cars/used/sale/vaz/2170/1106147770-c999a776/
https://auto.ru/cars/used/sale/vaz/2170/1106176941-51e79562/
https://auto.ru/cars/used/sale/vaz/2170/1106147720-5888f454/
https://a

ChunkedEncodingError: ignored

In [ ]:
!pip install imagededup

     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 17.3 MB 33.1 MB/s 
     |████████████████████████████████| 2.1 MB 49.7 MB/s 
     |████████████████████████████████| 4.4 MB 61.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: PyWavelets
    Found existing installation: PyWavelets 1.2.0
    Uninstalling PyWavelets-1.2.0:
      Successfully uninstalled PyWavelets-1.2.0
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires numpy>=1.17, but you have numpy 1.16.6 which is incompatible.
scikit-image 0.18.3 requires PyWavelet

In [ ]:
#Удалю дублирующиеся фотографии https://github.com/idealo/imagededup

from imagededup.methods import PHash
phasher = PHash()
for class_car in classes.keys():
  PATH_DATA = f'./drive/MyDrive/db_cars_VAZ/{class_car}/'
  duplicates = phasher.find_duplicates_to_remove(image_dir=PATH_DATA)
  print(f'Было в {class_car}:{len(os.listdir(PATH_DATA))}, из них дублируются {len(duplicates)}')
  for image in duplicates:
    os.remove(PATH_DATA+image)

2021-12-03 18:05:39,010: INFO Start: Calculating hashes...
100%|██████████| 584/584 [00:12<00:00, 48.40it/s]
2021-12-03 18:05:51,159: INFO End: Calculating hashes!
2021-12-03 18:05:51,163: INFO Start: Evaluating hamming distances for getting duplicates
2021-12-03 18:05:51,164: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 584/584 [00:00<00:00, 878577.31it/s]
2021-12-03 18:05:51,296: INFO End: Retrieving duplicates using Cython Brute force algorithm
2021-12-03 18:05:51,297: INFO End: Evaluating hamming distances for getting duplicates
2021-12-03 18:05:51,462: INFO Start: Calculating hashes...


Было в 0:584, из них дублируются 61


100%|██████████| 752/752 [00:33<00:00, 22.25it/s]
2021-12-03 18:06:25,360: INFO End: Calculating hashes!
2021-12-03 18:06:25,362: INFO Start: Evaluating hamming distances for getting duplicates
2021-12-03 18:06:25,367: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 752/752 [00:00<00:00, 61642.37it/s]
2021-12-03 18:06:25,496: INFO End: Retrieving duplicates using Cython Brute force algorithm
2021-12-03 18:06:25,498: INFO End: Evaluating hamming distances for getting duplicates


Было в 1:752, из них дублируются 186


2021-12-03 18:06:29,480: INFO Start: Calculating hashes...
100%|██████████| 737/737 [01:42<00:00,  7.19it/s]
2021-12-03 18:08:12,031: INFO End: Calculating hashes!
2021-12-03 18:08:12,032: INFO Start: Evaluating hamming distances for getting duplicates
2021-12-03 18:08:12,033: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 737/737 [00:00<00:00, 66007.60it/s]
2021-12-03 18:08:12,166: INFO End: Retrieving duplicates using Cython Brute force algorithm
2021-12-03 18:08:12,167: INFO End: Evaluating hamming distances for getting duplicates


...

Было в 7:1204, из них дублируются 118


2021-12-03 18:12:32,051: INFO Start: Calculating hashes...
100%|██████████| 737/737 [01:31<00:00,  8.09it/s]
2021-12-03 18:14:03,286: INFO End: Calculating hashes!
2021-12-03 18:14:03,288: INFO Start: Evaluating hamming distances for getting duplicates
2021-12-03 18:14:03,290: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 737/737 [00:00<00:00, 52817.59it/s]
2021-12-03 18:14:03,419: INFO End: Retrieving duplicates using Cython Brute force algorithm
2021-12-03 18:14:03,420: INFO End: Evaluating hamming distances for getting duplicates


Было в 8:737, из них дублируются 565


2021-12-03 18:14:04,664: INFO Start: Calculating hashes...
100%|██████████| 1380/1380 [00:26<00:00, 52.08it/s]
2021-12-03 18:14:31,247: INFO End: Calculating hashes!
2021-12-03 18:14:31,248: INFO Start: Evaluating hamming distances for getting duplicates
2021-12-03 18:14:31,253: INFO Start: Retrieving duplicates using Cython Brute force algorithm
100%|██████████| 1380/1380 [00:00<00:00, 73964.16it/s]
2021-12-03 18:14:31,386: INFO End: Retrieving duplicates using Cython Brute force algorithm
2021-12-03 18:14:31,389: INFO End: Evaluating hamming distances for getting duplicates


Было в 9:1380, из них дублируются 145


In [ ]:
DIR = './drive/MyDrive/db_cars_VAZ/'
dataset = []
for root, dirs, files in os.walk(DIR):  
    print(len(files)) 
    dataset.append(len(files))
print('Итого:',sum(dataset))

0
523
566
528
600
1086
172
1235
Итого: 4710
